#### 문제 설명  
모든 수가 0 또는 1로 이루어진 2차원 배열 a가 주어집니다. 다음 조건을 모두 만족하는 2차원 배열 b의 경우의 수를 ($10^7$ + 19)로 나눈 나머지를 return 하도록 solution 함수를 완성해주세요.  
  
- b의 모든 원소는 0 아니면 1입니다.  
- a의 행/열의 개수와 b의 행/열의 개수가 같습니다. (= a와 b의 크기가 같습니다.)  
- i = 1, 2, ..., (a의 열의 개수)에 대해서 a의 i번째 열과 b의 i번째 열에 들어 있는 1의 개수가 같습니다.  
  
b의 각 행에 들어 있는 1의 개수가 짝수입니다. (0도 짝수입니다.)  
  
#### 제한 사항  
- a의 행의 개수는 1 이상 300 이하입니다.  
    - a의 각 행의 길이는 1 이상 300 이하로 모두 동일합니다.  
  
#### 입출력 예  
|a	|result|
|:---:|:---:|
|[[0,1,0],[1,1,1],[1,1,0],[0,1,1]]|	6|
|[[1,0,0],[1,0,0]]|	0|
|[[1,0,0,1,1],[0,0,0,0,0],[1,1,0,0,0],[0,0,0,0,1]]|	72|

---

### 문제 풀이

In [1]:
def get_combination(combination, n, r):
    if combination[n][r] != -1:
        return combination[n][r]
    if n == 1 or r == 0 or n == r:
        combination[n][r] = 1
        return combination[n][r]
    combination[n][r] = get_combination(combination, n-1, r-1) + get_combination(combination, n-1, r)
    return combination[n][r]
    
def solution(a):
    row_len = len(a) # 행의 개수
    col_len = len(a[0]) # 열의 개수
    col = [0 for _ in range(col_len)] # 각 열당 1의 개수
    for x in range(col_len): # 각 열에 대한 1의 개수 갱신
        for y in range(row_len):
            if a[y][x] == 1:
                col[x] += 1
    combination = [[-1 for _ in range(row_len+1)] for _ in range(row_len+1)] # combination[n][r] = n C r의 개수
    even_count = [[-1 for _ in range(row_len+1)] for _ in range(col_len)]
    # even_count[a][b] = 0 ~ a열까지 짝수행의 개수가 b인 행렬의 개수
    for i in range(row_len+1):
        even_count[0][i] = 0 # 0열까지 짝수행의 개수가 i인 행렬의 개수는 0으로 갱신
    even_count[0][row_len-col[0]] = get_combination(combination, row_len, row_len-col[0])
    # 0번째 열의 경우, 0번째 열에서 가져야 하는 0의 개수(=총 행의 개수 - col[0])만큼의 짝수행을 지닐 수 있으므로 갱신
    answer = 0 # 답
    
    for by in range(1, col_len): # 0 ~ by번째까지의 열에 대해서
        for count in range(row_len+1): # 짝수행의 개수를 count만큼 지닌
            even_count[by][count] = 0 # 0 ~ by번째까지의 열에 대해서 짝수행의 개수를 count만큼 지닌 경우의 수
            for even in range(col[by]+1):
                odd = col[by] - even 
                # even: 0 ~ by-1번째 열까지로 구성된 행렬에서 1을 추가하여 홀수로 바꿀 (합이 짝수가 되는 행)의 개수
                # odd: 0 ~ by-1번째 열까지로 구성된 행렬에서 1을 추가하여 짝수로 바꿀 (합이 홀수가 되는 행)의 개수
                # even + odd = by번째 열에서의 1의 개수 (즉, 짝수/홀수 상태 변환 가능 개수)
                e = count + even - odd
                o = row_len - e
                # e: by-1번째 열까지로 구성된 행렬에서 e개의 짝수행을 지니고 있을 때
                # o: by-1번째 열까지로 구성된 행렬에서 o개의 홀수행을 지니고 있을 때
                # e + o = 전체 행의 개수
                if e < 0 or e > row_len or e < even or o < odd: # e가 행의 개수를 벗어나거나, even과 odd가 지니고 있는 행의 개수를 벗어나면
                    continue # 탐색하지 않음
                if even_count[by-1][e] == 0: # by-1번째 열까지로 구성된 행렬에서 e개의 짝수행을 지니고 있을 경우의 수가 없다면
                    continue # 탐색하지 않음
                even_count[by][count] += even_count[by-1][e] * get_combination(combination, e, even) * get_combination(combination, o, odd)
                # even_count[by][count]
                # = (by-1번째 열까지로 구성된 행렬에서 e개의 짝수행을 지니고 있을 경우의 수) * (짝수행 e개 중에 even개를 홀수행으로 바꿀 경우의 수) * (홀수행 o개 중에 odd개를 홀수행으로 바꿀 경우의 수) 
                even_count[by][count] %= (pow(10, 7) + 19) # (10^7 + 19)로 나눈 나머지값으로 저장
    answer = even_count[col_len-1][row_len] # 답은 모든 열을 포함하는 행렬에서 모든 행이 짝수일 경우의 수
    return answer # 답 반환

In [2]:
a = [[0,1,0],[1,1,1],[1,1,0],[0,1,1]]
solution(a)

6

In [3]:
a = [[1,0,0],[1,0,0]]
solution(a)

0

In [4]:
a = [[1,0,0,1,1],[0,0,0,0,0],[1,1,0,0,0],[0,0,0,0,1]]
solution(a)

72